In [1]:
import pymc as pm
print(pm.__version__)
import numpy as np
import matplotlib.pyplot as plt

5.12.0


In [2]:
import pandas as pd
import os
import sklearn
import tensorflow as tf

2025-01-05 22:13:56.055362: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Import data
path_to_data = '../data/ml-32m/'
print(path_to_data)
ratings = pd.read_csv(os.path.join(path_to_data, 'ratings.csv'))
movies = pd.read_csv(os.path.join(path_to_data, 'movies.csv'))

../data/ml-32m/


In [4]:
print(ratings.head(3), '\n')
print(movies.head(3))

   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976 

   movieId                    title  \
0        1         Toy Story (1995)   
1        2           Jumanji (1995)   
2        3  Grumpier Old Men (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  


In [5]:
print(ratings.describe())

             userId       movieId        rating     timestamp
count  3.200020e+07  3.200020e+07  3.200020e+07  3.200020e+07
mean   1.002785e+05  2.931861e+04  3.540396e+00  1.275241e+09
std    5.794905e+04  5.095816e+04  1.058986e+00  2.561630e+08
min    1.000000e+00  1.000000e+00  5.000000e-01  7.896520e+08
25%    5.005300e+04  1.233000e+03  3.000000e+00  1.051012e+09
50%    1.002970e+05  3.452000e+03  3.500000e+00  1.272622e+09
75%    1.504510e+05  4.419900e+04  4.000000e+00  1.503158e+09
max    2.009480e+05  2.927570e+05  5.000000e+00  1.697164e+09


In [6]:
# Check for missing values
print(ratings.isnull().sum())
print(movies.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
movieId    0
title      0
genres     0
dtype: int64


In [7]:
# Normalise data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
ratings['scaled_rating'] = scaler.fit_transform(ratings[['rating']])

In [8]:
# Merge dataframes to make dataframe containing ratings and genres
full_ratings = pd.merge(ratings, movies, on='movieId')

In [9]:
print("User Preferences Matrix:")
print(full_ratings.head(3), '\n')

User Preferences Matrix:
   userId  movieId  rating  timestamp  scaled_rating  \
0       1       17     4.0  944249077       0.777778   
1       1       25     1.0  944250228       0.111111   
2       1       29     2.0  943230976       0.333333   

                                               title  \
0                       Sense and Sensibility (1995)   
1                           Leaving Las Vegas (1995)   
2  City of Lost Children, The (Cité des enfants p...   

                                   genres  
0                           Drama|Romance  
1                           Drama|Romance  
2  Adventure|Drama|Fantasy|Mystery|Sci-Fi   



In [10]:
# Now group data by userID and genres, assign a mean rating for each (UserID, genre) pair then unpack these into a table
# Unfilled genres are filled with 0 rating

user_pref = full_ratings.groupby(['userId', 'genres']).rating.mean().unstack(fill_value=0)

In [11]:
# Now assume uniform prior for genres, use Dirichilet distribution as total prob sums to 1
with pm.Model() as model:
    user_genre_pref = pm.Dirichlet('user_genre_pref', a=np.ones(user_preferences.shape[1]))

NameError: name 'user_preferences' is not defined